# Setup

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!pip install -r /content/drive/MyDrive/RAG-LangChain/requirements.txt --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
Reason for being yanked: The Windows wheels, under some conditions, caused segfaults in unrelated user code.  Due to this we deleted the Windows wheels to prevent these segfaults, however this caused greater disruption as pip then began to try (and fail) to build 3.9.1 from the sdist on Windows which impacted far more users.  Yanking the whole release is the only tool available to eliminate these failures without changes to on the user side.  The sdist, OSX wheel, and manylinux wheels are all functional and there are no critical bugs in the release.   Downstream packagers shoul

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone as PineconeVectorStore
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from pinecone import Pinecone
import yaml
import os
import zipfile

In [8]:
with open('/content/drive/MyDrive/RAG-LangChain/config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
os.environ['PINECONE_API_KEY'] = config['PINECONE_API_KEY']
os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

# Busca Semântica

In [12]:
zip_file_path = '/content/drive/MyDrive/RAG-LangChain/documentos.zip'
extracted_folder_path = 'docs'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

documents = []
for filename in os.listdir(extracted_folder_path):
    print(filename)
    if filename.endswith(".pdf"):
        file_path = os.path.join(extracted_folder_path, filename)
        loader = PyMuPDFLoader(file_path)
        documents.extend(loader.load())

Disputa_Contratual_Empresa_W_contra_Fornecedor_K.pdf
Caso_de_Litígio_Comercial_entre_Empresas_A_e_B.pdf
Aç╞o_Civil_Pública_Município_contra_Empresa_de_Construç╞o.pdf
Processo_Penal_Estado_contra_Carlos_Pereira.pdf
Caso_de_Negligência_Médica_Ana_Oliveira_contra_Dr_Pedro_Sousa.pdf
Disputa_de_Terras_Fazenda_Boa_Vista_contra_Fazenda_Santa_Clara (1).pdf
Caso_de_Fraude_Financeira_Banco_Nacional_contra_Jo╞o_Almeida.pdf
Disputa_de_Propriedade_Intelectual_Empresa_X_contra_Empresa_Y.pdf
Aç╞o_Trabalhista_Funcionário_contra_Empresa_Z.pdf
Caso_de_Divórcio_e_Custódia_Jo╞o_Silva_e_Maria_Santos.pdf


In [13]:
print(documents)

[Document(metadata={'source': 'docs/Disputa_Contratual_Empresa_W_contra_Fornecedor_K.pdf', 'file_path': 'docs/Disputa_Contratual_Empresa_W_contra_Fornecedor_K.pdf', 'page': 0, 'total_pages': 1, 'format': 'PDF 1.3', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'PyPDF2', 'creationDate': '', 'modDate': '', 'trapped': ''}, page_content='Número do Processo: 822162\nAlegações de quebra de contrato pela Empresa W, que afirmou que o Fornecedor K não entregou\nos materiais conforme os termos acordados. Detalhes do contrato especificaram entregas mensais\nde materiais de construção de alta qualidade. O Fornecedor K argumentou que a Empresa W\nmudou as especificações sem aviso prévio. A resolução foi a favor da Empresa W, com o tribunal\nordenando ao Fornecedor K o pagamento de uma indenização de $300,000.\n'), Document(metadata={'source': 'docs/Caso_de_Litígio_Comercial_entre_Empresas_A_e_B.pdf', 'file_path': 'docs/Caso_de_Litígio_Comercial_entre_Empresas_

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len
)
chunks = text_splitter.create_documents([doc.page_content for doc in documents])

In [15]:
from pprint import pprint
pprint(chunks)

[Document(page_content='Número do Processo: 822162\nAlegações de quebra de contrato pela Empresa W, que afirmou que o Fornecedor K não entregou\nos materiais conforme os termos acordados. Detalhes do contrato especificaram entregas mensais\nde materiais de construção de alta qualidade. O Fornecedor K argumentou que a Empresa W\nmudou as especificações sem aviso prévio. A resolução foi a favor da Empresa W, com o tribunal\nordenando ao Fornecedor K o pagamento de uma indenização de $300,000.'),
 Document(page_content='Número do Processo: 539252\nDetalhes do contrato entre as empresas A e B. Motivos da disputa incluem alegações de quebra de\ncontrato, falta de pagamento e entrega de produtos defeituosos. A Empresa A alega que a\nEmpresa B não cumpriu os prazos acordados e forneceu produtos de qualidade inferior. A Empresa\nB argumenta que a Empresa A não fez os pagamentos conforme o contrato. A decisão judicial\nfavoreceu a Empresa A, ordenando que a Empresa B pague uma indenização de $5

In [16]:
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')

In [17]:
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7baae8f6ef20>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7baab39c8b20>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [18]:
index_name = 'llm'
vector_store = PineconeVectorStore.from_documents(chunks, embeddings, index_name=index_name)

In [19]:
query_1 = 'Responda apenas com base no input fornecido. Qual a Receita Total e Lucro Operacional no Quarto trimestre de 2023?'
query_2 = 'Responda apenas com base no input fornecido. Quais áreas devem ser fortalecidas de acordo com a auditoria interna? Foi encontrada alguma fraude?'
query_3 = 'Responda apenas com base no input fornecido. Quais os maiores ganhos de acordo com os investimentos da empresa?'
query_4 = 'Responda apenas com base no input fornecido. Quais os maiores riscos da empresa?'

In [20]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.2)

In [21]:
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

In [22]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever=retriever)

In [23]:
print(retriever)

tags=['Pinecone', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x7baab7322140> search_kwargs={'k': 3}


In [24]:
answer_1 = chain.invoke(query_1)
answer_2 = chain.invoke(query_2)
answer_3 = chain.invoke(query_3)
answer_4 = chain.invoke(query_4)

In [25]:
print('Pergunta: ',answer_1['query'])
print('Resultado: ',answer_1['result'],'\n')

Pergunta:  Responda apenas com base no input fornecido. Qual a Receita Total e Lucro Operacional no Quarto trimestre de 2023?
Resultado:  Com base nas informações fornecidas, não é possível determinar a Receita Total e o Lucro Operacional específicos do quarto trimestre de 2023. A projeção financeira para 2024 e o relatório da tesouraria se referem a dados e previsões para o ano de 2024, não fornecendo detalhes trimestrais para 2023. 



In [26]:
print('Pergunta: ',answer_2['query'])
print('Resultado: ',answer_2['result'],'\n')

Pergunta:  Responda apenas com base no input fornecido. Quais áreas devem ser fortalecidas de acordo com a auditoria interna? Foi encontrada alguma fraude?
Resultado:  De acordo com o relatório de auditoria interna de 2023, as áreas que devem ser fortalecidas são contas a pagar e recebíveis. Não foi identificada nenhuma fraude significativa durante as auditorias periódicas. 



In [27]:
print('Pergunta: ',answer_3['query'])
print('Resultado: ',answer_3['result'],'\n')

Pergunta:  Responda apenas com base no input fornecido. Quais os maiores ganhos de acordo com os investimentos da empresa?
Resultado:  De acordo com o relatório fornecido, os maiores ganhos da empresa foram provenientes dos investimentos em títulos de curto prazo, com uma expectativa de retorno de 5%. 



In [28]:
print('Pergunta: ',answer_4['query'])
print('Resultado: ',answer_4['result'])

Pergunta:  Responda apenas com base no input fornecido. Quais os maiores riscos da empresa?
Resultado:  Com base nas informações fornecidas, os maiores riscos da empresa estão relacionados a violações de normas ambientais durante projetos de construção, condições de trabalho inadequadas e demissões sem justa causa, bem como disputas contratuais que envolvem quebra de contrato, falta de pagamento e entrega de produtos defeituosos. Estes riscos podem resultar em multas, indenizações e danos à reputação da empresa.
